# Segmenting and Clustering Neighborhoods in Toronto_3


In [12]:

import numpy as np 
import pandas as pd 
! pip install jsonlib-python3
! pip install geopy
! pip install folium
from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
import jsonlib as json
import requests

print('Libraries imported.')

Libraries imported.


## Explore and cluster the neighborhoods in Toronto

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
Toronto_df= pd.read_csv('toronto_part2.csv')
for lat, lng, borough, Neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Defining Foursquare Credentials and Version

In [10]:
CLIENT_ID = "RTZAH4ZXP2V4ZLNZAB5ZXSPBK2OOOAXGKKFXDZZYLU4SV2WR"
CLIENT_SECRET = 'RD3YQIJJNT1E2D52SCQXOMYSP0OG21WPTK4NCPQLZHFUARVX'
VERSION = '20200906'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RTZAH4ZXP2V4ZLNZAB5ZXSPBK2OOOAXGKKFXDZZYLU4SV2WR
CLIENT_SECRET:RD3YQIJJNT1E2D52SCQXOMYSP0OG21WPTK4NCPQLZHFUARVX


In [11]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [17]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
4,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
5,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Swiss Chalet,43.768122,-79.190493,Restaurant
6,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Shoppers Drug Mart,43.770282,-79.185012,Pharmacy
7,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Shoeless Joe's,43.768892,-79.185851,Sports Bar
8,"Guildwood, Morningside, West Hill",43.765806,-79.185284,KFC,43.768900,-79.185600,Fast Food Restaurant
9,"Guildwood, Morningside, West Hill",43.765806,-79.185284,RBC Royal Bank,43.766790,-79.191151,Bank


In [18]:
toronto_venues.shape

(2275, 7)

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4
Woodbine Heights,13,13,13,13,13,13


## Analysing Each Neighborhood

In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_onehot.shape

(2275, 271)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Printing each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0    Breakfast Spot   0.2
1      Skating Rink   0.2
2       Supermarket   0.2
3  Sushi Restaurant   0.2
4   Badminton Court   0.2


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store   0.1
1        Coffee Shop   0.1
2     Sandwich Place   0.1
3        Pizza Place   0.1
4       Dance Studio   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.09
1            Bank  0.09
2  Sandwich Place  0.04
3     Supermarket  0.04
4     Bridal Shop  0.04


----Bayview Village----
                        venue  freq
0  Construction & Landscaping  0.33
1                 Flower Shop  0.33
2                       Trail  0.33
3               Movie Theater  0.00
4                Night Market  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1         Coffee Shop  0.10
2  Italian Restaurant  0.10
3           Juice Bar  0.05
4  

                        venue  freq
0           Convenience Store  0.25
1                  Playground  0.25
2                 Coffee Shop  0.25
3            Department Store  0.25
4  Modern European Restaurant  0.00


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.09
1                    Coffee Shop  0.07
2  Vegetarian / Vegan Restaurant  0.07
3             Mexican Restaurant  0.07
4                    Art Gallery  0.04


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
            venue  freq
0  Sandwich Place  0.12
1        Pharmacy  0.12
2            Bank  0.12
3    Liquor Store  0.12
4     Coffee Shop  0.12


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.20
1  Furniture / Home Store  0.05
2          Cosmetics Shop  0.04
3              Restaurant  0.04
4              Food Court  0.04


----Lawrence Park----
             

4       Night Market   0.0


----Willowdale, Newtonbrook----
                     venue  freq
0              Bridal Shop  0.33
1                     Café  0.33
2                     Park  0.33
3  North Indian Restaurant  0.00
4             Noodle House  0.00


----Willowdale, Willowdale East----
               venue  freq
0   Ramen Restaurant  0.08
1        Coffee Shop  0.06
2  Korean Restaurant  0.06
3               Café  0.04
4     Sandwich Place  0.04


----Willowdale, Willowdale West----
           venue  freq
0        Butcher  0.17
1           Bank  0.17
2    Pizza Place  0.17
3  Grocery Store  0.17
4           Park  0.17


----Woburn----
               venue  freq
0        Coffee Shop  0.50
1  Korean Restaurant  0.25
2       Soccer Field  0.25
3  Accessories Store  0.00
4       Noodle House  0.00


----Woodbine Heights----
                  venue  freq
0      Sushi Restaurant  0.08
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3                   Bar  0.08
4        

## Let's put that into a pandas dataframe
#### First, let's write a function to sort the venues in descending order.

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Supermarket,Badminton Court,Skating Rink,Sushi Restaurant,Field,Fast Food Restaurant,Farmers Market,Filipino Restaurant,Donut Shop
1,"Alderwood, Long Branch",Gas Station,Pizza Place,Pharmacy,Convenience Store,Sandwich Place,Gym,Athletics & Sports,Pub,Coffee Shop,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Mobile Phone Shop,Frozen Yogurt Shop,Fried Chicken Joint,Sushi Restaurant,Supermarket,Shopping Mall,Middle Eastern Restaurant
3,Bayview Village,Flower Shop,Construction & Landscaping,Trail,Event Service,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Restaurant,Juice Bar,Café,Thai Restaurant,Pub,Liquor Store


## Cluster Neighborhoods
### Run k-means to cluster the neighborhood into 5 clusters.

In [28]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,
       0, 4, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 3, 4, 0, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 4, 4, 4, 0, 0, 4, 0, 0, 4, 0, 0, 0,
       4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       0, 4, 0, 4, 0, 0, 0, 0, 3, 0])

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,0.0,Trail,Yoga Studio,Event Service,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Doctor's Office
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,0.0,Home Service,Golf Course,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm,Dog Run
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,0.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Mexican Restaurant,Pharmacy,Sports Bar,Fried Chicken Joint,Supermarket
3,M1G,Scarborough,Woburn,43.771545,-79.218135,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Farm,Dog Run
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,0.0,Gaming Cafe,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farm,Farmers Market


In [30]:
#We find that there is no data available for some neighbourhood droping that row

toronto_merged=toronto_merged.dropna()

In [31]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [33]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
### Cluster 1

In [34]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Trail,Yoga Studio,Event Service,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Doctor's Office
1,Scarborough,0,Home Service,Golf Course,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm,Dog Run
2,Scarborough,0,Pizza Place,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Mexican Restaurant,Pharmacy,Sports Bar,Fried Chicken Joint,Supermarket
3,Scarborough,0,Coffee Shop,Soccer Field,Korean Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Farm,Dog Run
4,Scarborough,0,Gaming Cafe,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farm,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...,...
96,North York,0,Furniture / Home Store,Yoga Studio,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farmers Market
99,Etobicoke,0,Pizza Place,Coffee Shop,Sandwich Place,Chinese Restaurant,Intersection,Discount Store,Creperie,Dog Run,Filipino Restaurant,Field
100,Etobicoke,0,Bank,Coffee Shop,Sandwich Place,Shopping Mall,Liquor Store,American Restaurant,Pharmacy,Pizza Place,Fast Food Restaurant,Farmers Market
101,Etobicoke,0,Caribbean Restaurant,Fast Food Restaurant,Hardware Store,Pharmacy,Video Store,Pizza Place,Grocery Store,Park,Beer Store,Japanese Restaurant


### Cluster 2

In [35]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1,Pharmacy,Yoga Studio,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm


### Cluster 3

In [36]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,2,Convenience Store,Yoga Studio,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farmers Market


### Cluster 4

In [37]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,3,Park,Convenience Store,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
93,Etobicoke,3,Park,Baseball Field,Skating Rink,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Yoga Studio
97,North York,3,Playground,Park,Event Service,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Doctor's Office


### Cluster 5

In [38]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,4,Bakery,Brewery,Wine Shop,Event Service,Falafel Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
17,North York,4,Park,Residential Building (Apartment / Condo),Dog Run,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Doctor's Office
21,North York,4,Park,Bridal Shop,Café,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Falafel Restaurant,Farm
25,North York,4,Food & Drink Shop,Construction & Landscaping,Park,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
41,East Toronto,4,Park,Grocery Store,Discount Store,Business Service,Bus Line,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Yoga Studio
45,Central Toronto,4,Breakfast Spot,Park,Food & Drink Shop,Department Store,Gym / Fitness Center,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
46,Central Toronto,4,Playground,Garden,Gym Pool,Park,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service
48,Central Toronto,4,Park,Lawyer,Gym,Tennis Court,Yoga Studio,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
50,Downtown Toronto,4,Playground,Candy Store,Grocery Store,Park,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Filipino Restaurant
59,Downtown Toronto,4,Theme Park,Fast Food Restaurant,Harbor / Marina,Farm,Park,Yoga Studio,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
